# Notebook for Part 1

This notebook contains the script and analysis for part 1 of the assessment 3. This takes the pre-processed data (i.e. output from the "UK Weather Data Preprocessing Script" notebook and conducts the relevant pre-processing and analysis relevant for part 1 of assessment 3.

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import datetime

os.environ["PROJ_LIB"] = 'C:\\Users\\angsi\\anaconda3\\pkgs\\proj4-5.2.0-ha925a31_1\\Library\\share'
from mpl_toolkits.basemap import Basemap

In [ ]:
# Load the data
proc_data_path = '..\proc_data\weather'
weather_df = pd.read_csv(os.path.join(proc_data_path,'weather.csv'))

# Exploratory Data Analysis

Check summary statistics of each weather related column for each station, mean, median, standard deviation

In [ ]:
weather_cols = ['tmax degC','tmin degC','af days', 'rain mm','sun hours']

In [ ]:
for station in weather_df['station'].unique():
    tmp_df = weather_df[weather_df['station'] == station]
    # Assign date
    tmp_df['date'] = tmp_df.apply(lambda x: datetime.datetime(x['yyyy'], x['mm'], 1), axis = 1)
    for weather_var in weather_cols:
        plt.figure(figsize = (15,8))
        plt.plot(tmp_df['date'],tmp_df[weather_var])
        plt.xlabel('Year')
        plt.ylabel(weather_var)
        plt.title('Plot of {}: {} Data Over Time'.format(station, weather_var))
        plt.show()

In [ ]:
# Looking at the mean does seem to provide a good indication of what might fall within the same cluster
weather_df.groupby('station')[weather_cols].mean()

In [ ]:
for weather_col in weather_cols:
    plt.figure()
    weather_df.groupby('station')[weather_col].mean().plot.bar()
    plt.title('Mean of {}'.format(weather_col))
    plt.show()

In [ ]:
# The coefficient of variation indicates that af days has more variation (relatively) across years 
# and it is worth investigating more
coeff_var_df = weather_df.groupby('station')[weather_cols].agg(np.std)/weather_df.groupby('station')[weather_cols].agg(np.mean)
coeff_var_df

In [ ]:
# Check for intra-year variation
intrayear_cv = weather_df.groupby(['station','yyyy'])[weather_cols].agg(np.std)/weather_df.groupby('station')[weather_cols].agg(np.mean)

# Seems like there is still quite a fair of variation intra-year for af days
intrayear_cv.mean()

In [ ]:
# Plotting the standard deviation
for weather_col in weather_cols:
    plt.figure()
    weather_df.groupby('station')[weather_col].std().plot.bar()
    plt.title('Standard Deviation of {}'.format(weather_col))
    plt.show()

In [ ]:
# Median provides a fairly similar view to the mean
weather_df.groupby('station')[weather_cols].median()

In [ ]:
for weather_col in weather_cols:
    plt.figure()
    weather_df.groupby('station')[weather_col].median().plot.bar()
    plt.title('Median of {}'.format(weather_col))
    plt.show()

Since this is a time series, we should look into monthly data to see how the weather data is like for each station. I think this is a lot more insightful and arguably more affirmative as seasonal changes of weather data would be captured.

In [ ]:
pd.options.display.max_rows = 999
weather_df.groupby(['station','mm'])[weather_cols].mean()

I will prepare the average monthly weather data for each station across all years

In [ ]:
X = weather_df.groupby(['station','mm'])[weather_cols].mean().reset_index()

X = X.pivot(index='station', columns='mm', values=weather_cols)
X.columns = [' '.join([col[0],str(col[1])]).strip() for col in X.columns.values]

In [ ]:
# Standardise X before clustering
scaler = StandardScaler()
X_standardised = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
X_standardised.head()

# Clustering

* K-means Clustering
* GMM Clustering

I start off this section by defining the relevant functions for kmeans clustering, GMM Clustering and plotting function visualise the stations on the map

In [ ]:
def kmeans_clustering(X, max_K):
    '''
    Takes X and max_K (max number of clusters) does kmeans clustering
    Returns the kmeans and inertia list
    '''
    kmean_lst = []
    inertia_lst = []
    for k in range(1,max_K):
        kmeans = KMeans(n_clusters=k, random_state=1000)
        kmeans.fit(X)
        inertia_lst.append(kmeans.inertia_)
        kmean_lst.append(kmeans)
    
    
    plt.figure()
    plt.plot(range(1,max_K), inertia_lst)
    plt.title('Inertia of the different clusters')
    plt.show()
        
    return kmean_lst, inertia_lst

In [ ]:
def gmm_clustering(X, max_K):
    '''
    Takes X and max_K (max number of clusters) does GMM clustering evaluating based on BIC and silhouette score
    Returns the kmeans and inertia list
    '''
    
    # Initialise GMM list, BIC list and silhouette score list
    gm_lst = []
    bic_lst = []
    silhouette_score_lst = []
    
    for k in range(1,max_K):
        
        # Fix random seed and fit GMM
        gm = GaussianMixture(n_components=k, random_state=0).fit(X)
        gm_lst.append(gm)
        
        # Get the labels
        labels = gm.predict(X)
        
        # Compute BIC and update BIC list
        bic_lst.append(gm.bic(X))
        
        # Compute silhouette score and list
        if k == 1:
            silhouette_score_lst.append(None)
        else:
            silhouette_score_lst.append(silhouette_score(X, labels, metric='euclidean'))
        
    # Plot BIC
    plt.figure()
    plt.plot(range(1,max_K), bic_lst)
    plt.title('BIC Scores')
    plt.show()
    
    # Plot Silhouette Score
    plt.figure()
    plt.plot(range(1,max_K), silhouette_score_lst)
    plt.title('Silhouette Scores')
    plt.show()
    
    return gm_lst, bic_lst, silhouette_score_lst


In [ ]:
# Identify what latitude and longitude could be useful for plotting by taking average lat long for the stations
weather_df.groupby('station')[['lat','lon']].mean().mean()

In [ ]:
def extract_rgba_from_cmap(cmap):
    '''
    Function to extract rgba from cmap
    '''
    rgba_lst = []
    iteration = 0
    while (True):
        if iteration == 0:
            rgba_lst.append(cmap(iteration))
        else:
            # When there is no change in cmap we break the while loop
            if cmap(iteration) == cmap(iteration-1):
                break
            else:
                rgba_lst.append(cmap(iteration))
        
        iteration += 1
        
    return rgba_lst


In [ ]:
def plot_ukmap(lat, lon, cluster_labels, station_names):
    fig = plt.figure(figsize=(12, 12))
    m = Basemap(projection='lcc', resolution='h',
                width=0.8E6, height=1.2E6, 
                lat_0=55, lon_0=-2.7,)
    m.etopo(scale=0.9, alpha=0.9)
    m.fillcontinents(color="#bbe2c6", lake_color='#c7dbfc')
    m.drawmapboundary(fill_color="#c7dbfc")
    m.drawcoastlines(color = 'gray')
    
    # If it goes beyond 17 cluster labels, this function doesn't support it.
    if len(np.unique(cluster_labels)) > 17:
        print('More than 17 clusters, colours not supported.')
        return None
    else:
        qualcmap1 = matplotlib.cm.get_cmap('Set1')
        qualcmap2 = matplotlib.cm.get_cmap('Set2')
        cmap_lst = extract_rgba_from_cmap(qualcmap1)
        cmap_lst.extend(extract_rgba_from_cmap(qualcmap2))

        color_lst = [cmap_lst[cluster_label] for cluster_label in cluster_labels]
        x, y = m(lon, lat)
        
        # Plot points
        m.scatter(lon, lat, latlon = True, marker='D',color = color_lst, s = 50, zorder=10)

        # Shift it a bit to the right and plot names
        for i, station_name in enumerate(station_names):
            plt.text(x[i]+20000, y[i], station_name, color = color_lst[i],fontsize = 8)
        
    plt.show()
    

In [ ]:
# Testing the function
plot_ukmap([53.781324], [-2.72], [10], ['Random Point'])

## K-means Clustering

In [ ]:
kmean_lst, inertia_lst =kmeans_clustering(X_standardised, 10)

### Picking the number of clusters for K-means clustering
Seems like 2 or 4 clusters are the "elbows"

In [ ]:
# Let's try 2
kmean_lst[1].labels_

Checking out the labels for 2 clusters

In [ ]:
X.index[kmean_lst[1].labels_ == 1]

In [ ]:
X.index[kmean_lst[1].labels_ == 0]

Checking out the labels for 4 clusters

In [ ]:
labels = kmean_lst[3].labels_

In [ ]:
for i in range(4):
    print('Cluster {} includes: {}'.format(i, X.index[labels == i].tolist()))

### Prepare for Plotting the K-Means Clustering Results

In [ ]:
# Get the station lat lon data
stn_latlon = weather_df[['station','lat','lon']].drop_duplicates().reset_index(drop = True)
stn_latlon.head()

In [ ]:
# Get the original X with stations
X_with_stations = X.reset_index()

# Stations with lat lon
X_with_stations = pd.merge(X_with_stations, stn_latlon, how= 'left', on = 'station')

# Keep only relevant columns for plotting
X_with_stations = X_with_stations[['station','lat','lon']]

# Assign Cluster Labels
X_with_stations['Kmeans 2-Cluster Labels'] = kmean_lst[1].labels_
X_with_stations['Kmeans 4-Cluster Labels'] = kmean_lst[3].labels_

In [ ]:
X_with_stations.head()

In [ ]:
# Visualise results
plot_ukmap(X_with_stations['lat'].values, X_with_stations['lon'].values, X_with_stations['Kmeans 2-Cluster Labels'].tolist(),
          X_with_stations['station'].values)

In [ ]:
plot_ukmap(X_with_stations['lat'].values, X_with_stations['lon'].values, X_with_stations['Kmeans 4-Cluster Labels'].tolist(),
          X_with_stations['station'].values)

### Repeating the same steps for just the average weather data

In [ ]:
X_ave = weather_df.groupby(['station'])[weather_cols].mean()

# Standardise before clustering
scaler = StandardScaler()
X_ave_standardised = pd.DataFrame(scaler.fit_transform(X_ave), columns = X_ave.columns)

kmean_lst_ave, inertia_lst_ave = kmeans_clustering(X_ave_standardised, 10)

In [ ]:
labels = kmean_lst[3].labels_
for i in range(4):
    print('Cluster {} includes: {}'.format(i, X_ave.index[labels == i].tolist()))

In [ ]:
# Get the original X with stations
X_ave_with_stations = X_ave.reset_index()

# Stations with lat lon
X_ave_with_stations = pd.merge(X_ave_with_stations, stn_latlon, how= 'left', on = 'station')

# Keep only relevant columns for plotting
X_ave_with_stations = X_ave_with_stations[['station','lat','lon']]

# Assign Cluster Labels
X_ave_with_stations['Kmeans 2-Cluster Labels'] = kmean_lst[1].labels_
X_ave_with_stations['Kmeans 4-Cluster Labels'] = kmean_lst[3].labels_

In [ ]:
plot_ukmap(X_ave_with_stations['lat'].values, X_ave_with_stations['lon'].values, X_ave_with_stations['Kmeans 2-Cluster Labels'].tolist(),
          X_ave_with_stations['station'].values)

In [ ]:
plot_ukmap(X_ave_with_stations['lat'].values, X_ave_with_stations['lon'].values, X_ave_with_stations['Kmeans 4-Cluster Labels'].tolist(),
          X_ave_with_stations['station'].values)

## GMM Clustering

In [ ]:
gm_lst, bic_lst, silhouette_score_lst =gmm_clustering(X_standardised, 10)

For BIC score, I would take the cluster corresponding to the lowest score and for the silhouette score, the highest. Both metrics point to having 2 clusters in the GMM.

In [ ]:
X_with_stations['GMM 2-Cluster Labels'] = gm_lst[1].predict(X_standardised)

In [ ]:
# Visualise results
plot_ukmap(X_with_stations['lat'].values, X_with_stations['lon'].values, X_with_stations['GMM 2-Cluster Labels'].tolist(),
          X_with_stations['station'].values)

It yields the same cluster as K-means clustering algorithm

Repeating the exercise with average X

In [ ]:
gm_lst2, bic_lst2, silhouette_score_lst2 =gmm_clustering(X_ave_standardised, 10)

Based on the silouhette score, we still go with 2 clusters but with BIC, we will pick 7 clusters.

In [ ]:
X_ave_with_stations['GMM 2-Cluster Labels'] = gm_lst2[1].predict(X_ave_standardised)
X_ave_with_stations['GMM 7-Cluster Labels'] = gm_lst2[6].predict(X_ave_standardised)

In [ ]:
# Visualise results
plot_ukmap(X_ave_with_stations['lat'].values, X_ave_with_stations['lon'].values, X_ave_with_stations['GMM 2-Cluster Labels'].tolist(),
          X_ave_with_stations['station'].values)

In [ ]:
# Visualise results
plot_ukmap(X_ave_with_stations['lat'].values, X_ave_with_stations['lon'].values, X_ave_with_stations['GMM 7-Cluster Labels'].tolist(),
          X_ave_with_stations['station'].values)

From these clusters, it does appear that 2-cluster output from K-means algorithm or GMM using the average station monthly weather data provides more interpretable results as it clearly splits clusters data into north and south of UK; and it is more likely for weather to be similar for regions nearer to each other.